In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [3]:
service = Service()

# if you want to get fancy with the options
# here the the ChromeOptions documentation: https://chromedriver.chromium.org/capabilities#h.p_ID_102
# here is the list of Chrome arguments: https://peter.sh/experiments/chromium-command-line-switches/
# there are a lot...
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True) # this keeps the window open post execution/errors/ changing frames (explained later)

prefs = {"download.default_directory": '/Users/justinpaul/Downloads',  # note need to change this
         "directory_upgrade": True}
chrome_options.add_experimental_option("prefs", prefs)

In [18]:
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://us.shein.com/')
WebDriverWait(driver, timeout=10)

#click out of coupon pop up
x_box = driver.find_element(By.CLASS_NAME, "icon-close").click()

input = 'shirt'

search_bar = driver.find_element(By.NAME, "header-search").send_keys(input)

search_enter = driver.find_element(By.CLASS_NAME, "suiiconfont-critical").click()

#driver.find_element(By.CSS_SELECTOR, '[aria-label="Size"]').click()

# view_more=driver.find_element(By.CLASS_NAME, "show-more-icon").click()
# view_more=driver.find_element(By.CLASS_NAME, "show-more-icon").click()
# view_more=driver.find_element(By.CLASS_NAME, "show-more-icon").click()
# view_more=driver.find_element(By.CLASS_NAME, "show-more-icon").click()
# view_more=driver.find_element(By.CLASS_NAME, "show-more-icon").click()

parent_size=driver.find_element(By.CSS_SELECTOR, "[aria-label=Size]")
view_more=parent_size.find_element(By.CLASS_NAME, "side-filter__item-viewMore").click()

In [ ]:
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'lxml')
